In [2]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import contractions
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob
import re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import ipywidgets
import swifter
import gensim
from gensim import corpora
import pickle
import os

In [41]:
glassdoor_data_dir = "/ifs/projects/amirgo-identification/glassdoor_data/"
glassdoor_reviews = pd.read_csv(os.path.join(glassdoor_data_dir, "glassdoor_reviews_only.csv")
glassdoor_lda_model_dir = "/ifs/projects/amirgo-identification/glassdoor_lda_model/"

In [3]:
pros = [doc.split(' ') for doc in glassdoor_reviews['pros_cleaned'].dropna().tolist()]
dictionary = corpora.Dictionary(pros)
corpus = [dictionary.doc2bow(doc) for doc in pros]
pickle.dump(corpus, open(os.path.join(glassdoor_lda_model_dir, 'corpus.pkl'), 'wb'))
dictionary.save(os.path.join(glassdoor_lda_model_dir, 'dictionary.gensim')

In [6]:
%%time
NUM_TOPICS = 10

ldamodel = gensim.models.LdaMulticore(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=5, workers=4)
ldamodel.save(os.path.join(glassdoor_lda_model_dir, 'model5.gensim'))

CPU times: user 44min 57s, sys: 9min 22s, total: 54min 19s
Wall time: 1h 17s


In [8]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.120*"environment" + 0.073*"friendly" + 0.049*"fun" + 0.047*"nice" + 0.039*"atmosphere" + 0.039*"management" + 0.035*"coworkers" + 0.033*"place" + 0.030*"team" + 0.029*"staff"')
(1, '0.171*"benefit" + 0.088*"pay" + 0.028*"salary" + 0.024*"time" + 0.024*"decent" + 0.023*"health" + 0.022*"competitive" + 0.021*"opportunity" + 0.019*"bonus" + 0.019*"insurance"')
(2, '0.092*"flexible" + 0.082*"hour" + 0.059*"schedule" + 0.048*"easy" + 0.042*"pay" + 0.036*"time" + 0.033*"job" + 0.017*"money" + 0.015*"pretty" + 0.013*"manager"')
(3, '0.030*"opportunity" + 0.026*"learn" + 0.026*"project" + 0.023*"new" + 0.022*"product" + 0.021*"lot" + 0.020*"technology" + 0.017*"industry" + 0.017*"interesting" + 0.016*"different"')
(4, '0.039*"training" + 0.033*"experience" + 0.031*"opportunity" + 0.025*"help" + 0.021*"career" + 0.016*"support" + 0.016*"learn" + 0.015*"student" + 0.014*"skill" + 0.013*"grow"')
(5, '0.021*"year" + 0.019*"time" + 0.017*"job" + 0.015*"day" + 0.014*"best" + 0.013*"like" + 0.

Topic Modeling Cons

In [98]:
cons = [doc.split(' ') for doc in glassdoor_reviews['cons_cleaned'].dropna().tolist()]
cons_dictionary = corpora.Dictionary(cons)
cons_corpus = [cons_dictionary.doc2bow(doc) for doc in cons]
pickle.dump(cons_corpus, open(os.path.join(glassdoor_lda_model_dir, 'cons_corpus.pkl'), 'wb'))
cons_dictionary.save(os.path.join(glassdoor_lda_model_dir, 'cons_dictionary.gensim'))

In [99]:
%%time
NUM_TOPICS = 10
ldamodel = gensim.models.LdaMulticore(cons_corpus, num_topics = NUM_TOPICS, id2word=cons_dictionary, passes=5, workers=8)
ldamodel.save(os.path.join(glassdoor_lda_model_dir, 'cons_glassdoor_model.gensim'))

CPU times: user 54min 47s, sys: 11min 50s, total: 1h 6min 38s
Wall time: 1h 11min 45s


In [100]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.107*"con" + 0.072*"think" + 0.036*"job" + 0.035*"time" + 0.034*"place" + 0.023*"bad" + 0.022*"far" + 0.020*"negative" + 0.018*"come" + 0.014*"love"')
(1, '0.035*"year" + 0.014*"new" + 0.012*"job" + 0.008*"position" + 0.007*"review" + 0.007*"business" + 0.007*"hire" + 0.006*"month" + 0.006*"client" + 0.006*"contract"')
(2, '0.072*"management" + 0.019*"manager" + 0.014*"leadership" + 0.012*"upper" + 0.012*"lack" + 0.011*"team" + 0.010*"staff" + 0.009*"poor" + 0.009*"care" + 0.008*"like"')
(3, '0.038*"office" + 0.017*"location" + 0.017*"system" + 0.017*"project" + 0.015*"issue" + 0.013*"need" + 0.013*"client" + 0.012*"area" + 0.010*"technology" + 0.010*"time"')
(4, '0.154*"pay" + 0.067*"low" + 0.055*"high" + 0.035*"salary" + 0.027*"management" + 0.024*"raise" + 0.023*"poor" + 0.020*"turnover" + 0.019*"rate" + 0.015*"benefit"')
(5, '0.033*"change" + 0.017*"need" + 0.017*"process" + 0.016*"growing" + 0.015*"thing" + 0.013*"fast" + 0.013*"time" + 0.012*"business" + 0.012*"team" + 0.01

Prediction on Survey Data

In [3]:
survey_data = pd.read_csv("/ifs/gsb/amirgo/spacespace/spacespace/Coco/analyses_data/survey_hr_topic_modeling.csv")
survey_data = survey_data.dropna(subset=['pros_cleaned', 'cons_cleaned'])
survey_data.reset_index(drop=True, inplace=True)

In [6]:
from gensim.models.ldamulticore import LdaMulticore
ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'model5.gensim'))
pros = [doc.split(' ') for doc in survey_data['pros_cleaned'].tolist()]
doc_vectors = [ldamodel.id2word.doc2bow(doc) for doc in pros]
doc_topics = [ldamodel[doc_vector] for doc_vector in doc_vectors]
rows = []
for doc in doc_topics:
    curr_row = [0] * 10
    for topic, prop in doc:
        curr_row[topic] = prop
    rows.append(curr_row)
pros_df = pd.DataFrame(rows, columns=['pros_topic'+str(i) for i in range(1,11)])

In [8]:
ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'cons_glassdoor_model.gensim'))
cons = [doc.split(' ') for doc in survey_data['cons_cleaned'].tolist()]
doc_vectors = [ldamodel.id2word.doc2bow(doc) for doc in cons]
doc_topics = [ldamodel[doc_vector] for doc_vector in doc_vectors]
rows = []
for doc in doc_topics:
    curr_row = [0] * 10
    for topic, prop in doc:
        curr_row[topic] = prop
    rows.append(curr_row)
cons_df = pd.DataFrame(rows, columns=['cons_topic'+str(i) for i in range(1,11)])


In [9]:
pros_df.reset_index(drop=True, inplace=True)
cons_df.reset_index(drop=True, inplace=True)
survey_data = pd.concat([survey_data, pros_df, cons_df], axis=1)


In [11]:
survey_data.to_csv('/ifs/gsb/amirgo/spacespace/spacespace/Coco/analyses_data/coco_glassdoor_topic_modeling.csv', index=False)

Glassdoor Document Proportions 

In [42]:
glassdoor_reviews = glassdoor_reviews.dropna(subset=['pros_cleaned', 'cons_cleaned'])

In [32]:
from gensim.models.ldamulticore import LdaMulticore
ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'model5.gensim'))
# redoing this without loading saved corpus as the original models were not 
pros = [doc.split(' ') for doc in glassdoor_reviews['pros_cleaned'].tolist()]
doc_vectors = [ldamodel.id2word.doc2bow(doc) for doc in pros]
doc_topics = [ldamodel[doc_vector] for doc_vector in doc_vectors]
rows = []
for doc in doc_topics:
    curr_row = [0] * 10
    for topic, prop in doc:
        curr_row[topic] = prop
    rows.append(curr_row)
glassdoor_pros_df = pd.DataFrame(rows, columns=['pros_topic'+str(i) for i in range(1,11)])

In [33]:
ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'cons_glassdoor_model.gensim'))
cons = [doc.split(' ') for doc in glassdoor_reviews['cons_cleaned'].tolist()]
doc_vectors = [ldamodel.id2word.doc2bow(doc) for doc in cons]
doc_topics = [ldamodel[doc_vector] for doc_vector in doc_vectors]
rows = []
for doc in doc_topics:
    curr_row = [0] * 10
    for topic, prop in doc:
        curr_row[topic] = prop
    rows.append(curr_row)
glassdoor_cons_df = pd.DataFrame(rows, columns=['cons_topic'+str(i) for i in range(1,11)])

In [64]:
glassdoor_pros_df.reset_index(drop=True, inplace=True)
glassdoor_cons_df.reset_index(drop=True, inplace=True)
glassdoor_reviews.reset_index(drop=True, inplace=True)
glassdoor_reviews.drop(['pros_cleaned', 'cons_cleaned', 'Unnamed: 0'], axis=1, inplace=True)

In [77]:
glassdoor_reviews = pd.concat([glassdoor_reviews, glassdoor_pros_df, glassdoor_cons_df], axis=1)
glassdoor_reviews.to_csv(os.path.join(glassdoor_data_dir, 'glassdoor_topic_prop.csv'), index=False)

In [86]:
glassdoor_pros_ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'model5.gensim'))
pros_names = ['pros_topic'+str(i) for i in range(1,11)]
topics = glassdoor_pros_ldamodel.show_topics(formatted=False)
rows = []
for i in range(0, len(topics)):
    row = [pros_names[i]]
    word_prob_list = topics[i][1]
    for word, _ in word_prob_list:
        row.append(word)
    rows.append(row)

In [97]:
pros_top_words = pd.DataFrame(rows)
pros_top_words.set_index(0, inplace=True)
pros_top_words.to_csv(os.path.join(glassdoor_data_dir, 'glassdoor_pros_top_words_lda_k10.csv'))

In [99]:
glassdoor_cons_ldamodel = LdaMulticore.load(os.path.join(glassdoor_lda_model_dir, 'cons_glassdoor_model.gensim'))
cons_names = ['cons_topic'+str(i) for i in range(1,11)]
topics = glassdoor_cons_ldamodel.show_topics(formatted=False)
rows = []
for i in range(0, len(topics)):
    row = [cons_names[i]]
    word_prob_list = topics[i][1]
    for word, _ in word_prob_list:
        row.append(word)
    rows.append(row)
cons_top_words = pd.DataFrame(rows)
cons_top_words.set_index(0, inplace=True)
cons_top_words.to_csv(os.path.join(glassdoor_data_dir, 'glassdoor_cons_top_words_lda_k10.csv'))

In [110]:
glassdoor_reviews = pd.read_csv(os.path.join(glassdoor_data_dir, "glassdoor_reviews_only.csv"))
glassdoor_reviews = glassdoor_reviews.dropna(subset=['pros_cleaned', 'cons_cleaned'])
glassdoor_pros_df.reset_index(drop=True, inplace=True)
glassdoor_cons_df.reset_index(drop=True, inplace=True)
glassdoor_reviews.reset_index(drop=True, inplace=True)
glassdoor_reviews = pd.concat([glassdoor_reviews, glassdoor_pros_df, glassdoor_cons_df], axis=1)

In [119]:
glassdoor_original = pd.read_csv(os.path.join(glassdoor_data_dir, 'reviews_new_processed.csv'))

/opt/jupyter/hub/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [120]:
pd.set_option('max_colwidth', 400)
cons_t1 = glassdoor_reviews[['review_id','cons_cleaned', 'cons_topic1']].sort_values('cons_topic1', ascending=False).head(15)
cons_t2 = glassdoor_reviews[['review_id','cons_cleaned', 'cons_topic2']].sort_values('cons_topic2', ascending=False).head(15)
cons_t4 = glassdoor_reviews[['review_id','cons_cleaned', 'cons_topic4']].sort_values('cons_topic4', ascending=False).head(15)

In [124]:
cons_t1.merge(glassdoor_original[['review_id', 'cons']], on='review_id')[['cons_topic1', 'cons']]

,cons_topic1,cons
0,0.959091,No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.No cons.
1,0.957141,Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy Busy
2,0.952632,"No cons ,No cons,No cons,No cons ,No cons,No cons,No cons ,No cons,No cons,No cons ,No cons,No cons,No cons ,No cons,No cons,No cons ,No cons,No cons"
3,0.947059,No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons! No cons!
4,0.943750,No Cons No Cons No Cons No Cons No Cons No No Cons No Cons No Cons Cons No Cons No Cons No Cons No Cons No Cons No Cons
5,0.943749,fun fun fun fun fun fun fun fun funf fun fun fun fnunf fn fun fun fun fun
6,0.943742,No cons. Everything about this place is super super super super super super super super super super super super super good
7,0.940000,no cons no cons no cons no cons no cons no cons no cons\r\nno cons no cons no cons no cons no cons no cons no cons
8,0.939996,Absolutely no cons. Absolutely no cons Absolutely no cons Absolutely no cons Absolutely no cons Absolutely no cons Absolutely no cons
9,0.939972,nothing in particular that comes to mind. had a very positive experience so far and would recommend PwC to every person who wants to apply and looking for a consulting experience


In [125]:
cons_t2.merge(glassdoor_original[['review_id', 'cons']], on='review_id')[['cons_topic2', 'cons']]

,cons_topic2,cons
0,0.975662,"They don't actually hire enough people, they have more temps than actual hired in employees! They go through temporary employees through Adecco. Even if a temp has been there a year they won't just hire you in! \n\nInstead they bring in a new temp to take your place. So they constantly bring in new temps instead of hiring. \n\nWhen one temp is done in comes a new temp, then when that temp is d..."
1,0.972686,"Multiple layoffs in all departments. No raise for a span of two years. Bonuses eliminated 3 or 4 years ago. Never know if the company will lose money and downsize the staff more. Layoffs and steady outflow of talented employees have kept the company afloat. New product development has slowed significantly and innovation has all but stopped. Beware, there are not many alternate employers in..."
2,0.970959,A dead end job is a dead end job is a dead end job is a dead end job is a dead end job is a dead end job is a dead end job is a dead end job is a dead end job is a dead end job.
3,0.969972,"No stability, constant risk of losing your job; often for no apparent reason. The entire strategy going forward is to acquire new companies so do not expect any innovative work. The IT team is strangely obsessed with Microsoft products and costs the company huge amounts of money to use them vs cheaper alternatives."
4,0.968951,"Company went downhill after it was acquired by a private equity firm. No raises or bonuses - even for those that brought in $$$ via Business Development. Everything is about profit, profit, profit. Not focused on employee professional development. They just want young, cheap, barely qualified defense contractors. I don't see any future changes even with the new takeover."
5,0.968951,Shrinking positions. Layoffs constantly. Bonus pool is zero. Outsourcing jobs to India. They said compliance layoffs were done but more compliance folks were laid off. My team of 5 is down to just me and they don’t have the budget to fill those roles. Currently looking for new jobs. Stay away from state street as much as you can.
6,0.967834,The institute is struggling with a vision of where to go as its sales have dropped dramatically and cash flow has become a major concern. Major layoffs have occurred and will continue to occur for the foreseeable future. Beware if you accept a position here it is likely a very unstable situation and new hires are especially at risk.
7,0.967814,Athena is not the same company it was when I started a few years ago. The Elliot investment and buyout has completely shredded any culture that was left. All decisions at this point come down to cost saving and making the company more attractive to go public again or a new buyer in a year or two down the road. It's really disappointing to see a company that once had a great culture and mission...
8,0.966644,"Not all WF business lines have the same talent development pipeline methodology, and in some of the offshoot businesses such as consumer lending, new hires are often outside the company. Company values experience over education, so most mid tier and senior level positions require 10+ years of direct and or/related experience"
9,0.966640,"No Compliance to the State of Texas\r\nNo employee documents signed for Income Tax Records or Unemployment.\r\nWhen agree to take lay-off, the result was a termination notice.\r\nNo lay-off notices upon request after request \r\nCEO or executive have no idea how to run a legal business in the State of Texas."


In [127]:
cons_t4.merge(glassdoor_original[['review_id', 'cons']], on='review_id')[['cons_topic4', 'cons']]

,cons_topic4,cons
0,0.959080,"Parking was a problem due to having outgrown their facility and requiring new staff to park off site and walk in, which is quite a walk as it's not in the best area.\r\nComputer equipment and design apps are aging and need updating."
1,0.959054,"It can be a bit disorganized at times, and having multiple offices in various time zones introduces communication challenges. Communications equipment needs improvements, especially in light of how critical it is since we rely on remote communication so much."
2,0.957133,"Closed local office, closed local office, closed local office, closed local office, closed local office, closed local office, closed local"
3,0.957133,"Some of the projects are less interesting than others. It's possible to get ""stuck"" on a project, but the company is usually pretty good about moving you if you do not like the project. Sometimes clients can be difficult to deal with, but that is the nature of client work. Sometimes the open space office can be a bit loud."
4,0.957123,location is a little far to access on public transportation as it requires a shuttle from the BART- No direct Bart access- The open office environment can make it difficult for projects requiring concentration and quiet.
5,0.957068,"The current Bethlehem office is a bit tight, but we're moving to a larger space soon. There's also probably always going to be the regular issues of being in a small remote office and dealing with coworkers in another time zone."
6,0.954976,"We could use a better CRM, and better hardware so we could work remote sometimes. Desktop instead of Laptops sometimes makes it feel like your locked to your desk. It would be nice to have the ability to work in different spaces in the office."
7,0.954968,"Sunnyvale location is not good for public transit. It's possible to take caltrain, switch to light rail, then walk. You might as well drive which is never pleasant in the bay area. Best to live near the office if possible."
8,0.952626,None I'm new etc etc etc etc etc etc etc etc etc etc etc etc etc etc etc etc etc
9,0.952616,"Basic supplies are not always provided by the facility, empty space in the building that could be allocated more efficiently, lots of running up and down floors in order to work with clients, no central office to meet clients for my department"
